In [146]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from sklearn.metrics.pairwise import euclidean_distances
import math
from scipy.spatial import ConvexHull
import torch
import itertools


def plot_triangulation (points, triang):
    plt.triplot (points[:,0], points[:,1], triang)
    plt.plot (points[:,0], points[:,1], 'o')
    plt.show()

def calc_L (points, simplices):
    #simplices = Delaunay (points).simplices
    #distances = euclidean_distances (points, points)
    
    L = torch.zeros((len(points), len(points)))

    for sim in simplices:
        for ind in [[0, 1], [1, 2], [0, 2]]:
            #L [sim [ind [0]], sim [ind [1]]] = distances [sim [ind [0]], sim [ind [1]]]
            #L [sim [ind [1]], sim [ind [0]]] = distances [sim [ind [1]], sim [ind [0]]]
            L [sim [ind [0]], sim [ind [1]]] = ((points[sim [ind [0]]][0]-points[sim [ind [1]]][0])**2+\
                                               (points[sim [ind [0]]][1]-points[sim [ind [1]]][1])**2)**0.5
            L [sim [ind [1]], sim [ind [0]]] = ((points[sim [ind [0]]][0]-points[sim [ind [1]]][0])**2+\
                                               (points[sim [ind [0]]][1]-points[sim [ind [1]]][1])**2)**0.5

    return L

def calc_triangle_area (L, i1, i2, i3):
    l1 = L [i1, i2]
    l2 = L [i2, i3]
    l3 = L [i3, i1]

    p = (l1 + l2 + l3) / 2

    return math.sqrt (p * (p - l1) * (p - l2) * (p - l3))

def calc_A (simplices, L, points_num):
    A = torch.zeros ((points_num, points_num))
    
    for i in range (points_num):
        area = 0
        
        for j in range (len (simplices)):
            if (i in simplices [j]):
                sim = simplices [j]
                
                area_part = calc_triangle_area (L, sim [0], sim [1], sim [2])
                
                area += area_part / 3
        
        A [i, i] = area
    
    return A

def find_E_Eb (points, simplices):
    E = []
    
    for sim in simplices:
        E.append (sorted ((sim [0], sim [1])))
        E.append (sorted ((sim [1], sim [2])))
        E.append (sorted ((sim [2], sim [0])))
    
    E_b = []
    hull = ConvexHull (points).vertices
    
    for i in range (len (hull) - 1):
        E_b.append (sorted ((hull [i], hull [i + 1])))
    
    E_b.append (sorted ((hull [0], hull [-1])))
    
    #АЛАРМА
    #Тут нужно оставить в E и E_b только уникальные таплы.
    #Я попробовал это сделать, но с unhashable type какая-то морока, так то я забил.
    #В этом месте числа внутри тапла отсортированы.
    #Пробовал вот таким образом:
    #return list (set (E)), list (set (E_b))
    
    return E, E_b

def calc_W (E, E_b, A, L, simplices):
    W = torch.zeros (A.shape)
    
    sh = W.shape
    
    for i in range (sh [0]):
        for j in range (i + 1, sh [0]):
            kh_list = []
    
            for sim in simplices:
                elem = set (sim)                
                ele = set ([i, j])
                
                #print ("ele, elem")
                #print (ele, elem)
                
                if (ele.issubset (elem)):
                    kh_list.append (elem.difference (ele))
            
            if (len (kh_list) == 0):
                continue
            
            h = list (kh_list [0]) [0]
            
            if ([i, j] in E and [i, j] not in E_b):
                k = list (kh_list [1]) [0]
                
                val = 0
                
                val += (- L [i, j]**2 + L [j, k]**2 + L [k, i]**2) / \
                    (8 * calc_triangle_area (L, i, j, k))

                val += (- L [i, j]**2 + L [j, h]**2 + L [h, i]**2) / \
                    (8 * calc_triangle_area (L, i, j, h))
                
                W [i, j] = val
                
            elif ([i, j] in E_b):
                val = 0
                
                val += (- L [i, j]**2 + L [j, h]**2 + L [h, i]**2) / \
                    (8 * calc_triangle_area (L, i, j, h))
                
                W [i, j] = val
        
    for i in range (sh [0]):
        for j in range (i, sh [0]):
            W [j] [i] = W [i] [j]
    
    for i in range (sh [0]):
        W [i, i] = - sum (W [i, :])

    return W

def find_eigenvalues (W, A):
    product = torch.inverse (A) @ W
    
    lambdas, vectors = tuple (torch.symeig (product, eigenvectors=True))
    
    return lambdas#, vectors

def cut (val):
    return (min (0, val))**2

def rho (V, L, E_b, simplices):
    rho_1 = 0
    
    for e in E_b:
        rho_1 += L [e [0], e [1]]**2
    
    rho_2 = 0
    
    for s in simplices:
        for sim in list(itertools.permutations(s)):

            rot_matr = torch.tensor ([[0., -1.], [1., 0.]])
            sub_1 = V [sim [1]] - V [sim [0]]
            sub_2 = V [sim [2]] - V [sim [0]]

            curr_val = (rot_matr @ sub_1).transpose(0, -1) @ sub_2

            rho_2 += curr_val
    
    rho = rho_1 + cut (rho_2)
    
    return rho

def weighted_norm (a, b):
    norm = 0
    
    for i in range (len (a)):
        norm += (a [i] - b [i])**2 / (i + 1)
    
    return norm

initial = [(0.0, 0.0), (1.0, 0.0), (2.0, 1.0), (0.0, 2.0)]
points = torch.tensor(initial, requires_grad=True)

#points = np.array([[0, 0], [1, 0], [2, 0], [0, 2]])
    
#points = np.random.rand (6, 2)

print (points)

tri = Delaunay (initial)
simplices = tri.simplices

L = calc_L (points, simplices)
#print (L)

A = calc_A (simplices, L, len (points))

#print (A)

E, E_b = find_E_Eb (initial, simplices)

#print (E)
#print ("\n")
#print (E_b)

W = calc_W (E, E_b, A, L, simplices)

print (W)

pen = rho (points, L, E_b, simplices)

print (pen)

#plot_triangulation (points, simplices)

tensor([[0., 0.],
        [1., 0.],
        [2., 1.],
        [0., 2.]], requires_grad=True)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor(0., grad_fn=<SelectBackward>)
tensor([[-1.2500,  1.0000,  0.0000,  0.2500],
        [ 1.0000, -1.8333,  0.6667,  0.1667],
        [ 0.0000,  0.6667, -0.8333,  0.1667],
        [ 0.2500,  0.1667,  0.1667, -0.5833]], grad_fn=<CopySlices>)
tensor(12., grad_fn=<AddBackward0>)


In [90]:
find_eigenvalues(W, A).sum().backward()

In [91]:
points.grad

tensor([[ 0.3000,  0.6000],
        [-1.5000,  3.4000],
        [-0.4667, -0.2667],
        [ 1.6667, -3.7333]])

In [148]:
initial = [(0.0, 0.0), (1.0, 0.0), (2.0, 1.0), (0.0, 2.0)]

target = torch.tensor(initial)
tri = Delaunay (initial)
simplices = tri.simplices
E, E_b = find_E_Eb (initial, simplices)
L = calc_L (target, simplices)
A = calc_A (simplices, L, len (points))
W = calc_W (E, E_b, A, L, simplices)
mu = find_eigenvalues(W, A)
print (E, E_b)

tensor(0.)
tensor(0.)
tensor(0.)
tensor(0.)
[[1, 3], [0, 3], [0, 1], [1, 3], [1, 2], [2, 3]] [[0, 3], [0, 1], [1, 2], [2, 3]]


In [145]:
#initial = [(0.0, 0.0), (1.0, 0.0), (1.0, 1.0), (0.0, 1.0)]
initial = [(0.0, 0.0), (1.0, 0.0), (2.0, 1.0), (0.0, 1.0)]

points = torch.tensor(initial, requires_grad=True)
optimizer = torch.optim.Adam([points], lr = 0.000001)
tri = Delaunay (initial)
simplices = tri.simplices
E, E_b = find_E_Eb (initial, simplices)

for _ in range (10000):
    L = calc_L (points, simplices)
    A = calc_A (simplices, L, len (points))
    W = calc_W (E, E_b, A, L, simplices)
    pen = rho (points, L, E_b, simplices)
    
    current_eigen = find_eigenvalues(W, A)
    
    #loss =   weighted_norm(current_eigen, mu)+pen# calculate loss
    loss =   weighted_norm(current_eigen, mu)+pen# calculate loss

    optimizer.zero_grad()  # clear previous gradients
    loss.backward()        # compute gradients of all variables wrt loss

    optimizer.step()
    print (current_eigen)
    #print (loss)

tensor([-8.5887, -3.7713, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5887, -3.7714, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5888, -3.7714, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5888, -3.7714, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5888, -3.7714, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5889, -3.7714, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5889, -3.7714, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5889, -3.7715, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5890, -3.7715, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5890, -3.7715, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5890, -3.7715, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5891, -3.7715, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5891, -3.7715, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor([-8.5891, -3.7716, -1.3450,  1.2051], grad_fn=<SymeigBackward>)
tensor

tensor([-8.5929, -3.7734, -1.3453,  1.2057], grad_fn=<SymeigBackward>)
tensor([-8.5930, -3.7734, -1.3453,  1.2057], grad_fn=<SymeigBackward>)
tensor([-8.5930, -3.7734, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5930, -3.7734, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5931, -3.7734, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5931, -3.7734, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5931, -3.7735, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5932, -3.7735, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5932, -3.7735, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5932, -3.7735, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5933, -3.7735, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5933, -3.7735, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5933, -3.7736, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor([-8.5934, -3.7736, -1.3453,  1.2058], grad_fn=<SymeigBackward>)
tensor

tensor([-8.5971, -3.7754, -1.3455,  1.2064], grad_fn=<SymeigBackward>)
tensor([-8.5972, -3.7754, -1.3455,  1.2064], grad_fn=<SymeigBackward>)
tensor([-8.5972, -3.7754, -1.3455,  1.2064], grad_fn=<SymeigBackward>)
tensor([-8.5972, -3.7754, -1.3455,  1.2064], grad_fn=<SymeigBackward>)
tensor([-8.5973, -3.7754, -1.3455,  1.2064], grad_fn=<SymeigBackward>)
tensor([-8.5973, -3.7754, -1.3455,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5973, -3.7755, -1.3455,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5974, -3.7755, -1.3455,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5974, -3.7755, -1.3455,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5974, -3.7755, -1.3456,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5975, -3.7755, -1.3456,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5975, -3.7755, -1.3456,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5975, -3.7756, -1.3456,  1.2065], grad_fn=<SymeigBackward>)
tensor([-8.5976, -3.7756, -1.3456,  1.2065], grad_fn=<SymeigBackward>)
tensor

tensor([-8.6011, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6011, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6012, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6012, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6012, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6013, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6013, -3.7773, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6013, -3.7774, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6014, -3.7774, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6014, -3.7774, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6014, -3.7774, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6015, -3.7774, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6015, -3.7774, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor([-8.6015, -3.7775, -1.3458,  1.2071], grad_fn=<SymeigBackward>)
tensor

KeyboardInterrupt: 